-------------------------

 ## SER 347 - ST METRICS

--------------------------




Ouvintes: José Guilherme Fronza e Michel Eustáquio Dantas Chaves 

Data: 08/06/2020

## Sobre o ST METRICS

##### The stmetrics, is a python package that provides the extraction of state-of-the-art time-series features. These features can be used for remote sensing time-series image classification and analysis.

01: Instalação do pacote [stmetrics](https://stmetrics.readthedocs.io/en/latest/index.html) (Soares et al., 2020). Este pacote permite a extração de métricas fenológicas de classes de uso e cobertura da terra avaliadas no espaço e no tempo.

In [ ]:
#!pip install git+https://github.com/andersonreisoares/stmetrics@spatial --upgrade

02: Importação de bibliotecas utilizando comandos 'import' do ambiente Jupyter correspondente ao ambiente criado, considerando o Kernel específico para este ambiente.

In [ ]:
###Importing libs that we'll use to run your code
# Python Native
import os
import multiprocessing as mp
# 3rd party
import gdal
import rasterio
import stmetrics
import numpy
import pandas
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import shape
import fiona
from rasterio.mask import mask
import warnings
warnings.filterwarnings('ignore')

#how to get help in stmetrics
#help(stmetrics.basics.amplitude)


03: Definição de uma função para listar todas as imagens contidas na pasta padrão dos dados de entrada. Ao todo, foram consideradas 36 imagens entre os anos de 2017 e 2019.

In [ ]:
# Function to list all images from input path
def list_all_images(image_path):
    # Creates an empty list
    li_bands = []
    for name in os.listdir(image_path):
            li_bands.append(os.path.join(image_path,name))
    li_bands.sort()
    return li_bands

04: Obtenção da leitura e listagem das imagens contidas na pasta padrão dos dados de entrada. Output: leitura e listagem dentro do ambiente Jupyter.


In [ ]:
# List input images in folder to stack over time 2017~2019 - 36 images

#input path folder 
path = 'C:/path_images'

#run function created above to list input path folder
listimg = list_all_images(path)

#print total image in list to check
print(len(listimg))

#print images names - list elements - to check
print(listimg)

05: Definição de pasta padrão para o armazenamento do arquivo stack gerado a partir de toda a série de imagens contidas na pasta padrão dos dados de entrada. Output: leitura das 36 imagens e mosaico de todas elas.


In [ ]:
# Input output path to image data stack
path_out = 'C:/path'

#set working output dir
os.chdir(path_out)

# Read metadata of first file
with rasterio.open(listimg[0]) as src0:
    #read metadata of first file in the list and put into "meta"
    meta = src0.meta
    
# Update "meta" to get total list items
meta.update(count = len(listimg))

# Read each layer and write it to stack
with rasterio.open('lc8_2017_2019_stack.tif', 'w', **meta) as dst:
    #loop to list images in listimg
    for id, layer in enumerate(listimg, start=1):
        print(id, layer)
        #open the "layer" as src1
        with rasterio.open(layer) as src1:
            #write it in dst temp file
            dst.write_band(id, src1.read(1))

06: Determinação de comando utilizando a biblioteca gdal para condicionar a projeção da imagem stack para a mesma projeção dos dados amostrais, EPSG:32723. Ainda nesta etapa, foi definida a pasta padrão para alocar o arquivo com nova projeção.

In [ ]:
### Using gdal warp to reproject stack tif

# Set input file name to warp
filename = r"/path/lc8_2017_2019_stack.tif"

# Open the input file with gdal.Open
input_raster = gdal.Open(filename)

# Set output file name
output_raster = r"/path/lc8_2017_2019_stack_32723R.tif"

# Warp the input tif to selected EPSG in dstSRS
gdal.Warp(output_raster,input_raster,dstSRS='EPSG:32723')

# Clean memory
del input_raster
del output_raster

07: Leitura dos arquivos de amostras, em formato shapefile (.shp), e seleção de talhões específicos para análise no stmetrics.

In [ ]:
## Open shapefile geometry to select id from list
shapefile_path = '/path/samples.shp'

#select list shapefile ID to crop and input in STMETRICS
list_ids = [13, 148, 64, 32,189, 193, 121]

#output path to selected id shapefile
path_out = '/path/'

#set working output dir
os.chdir(path_out)        

#fiona to open shapefile, write meta variable
with fiona.open(shapefile_path, "r") as entrada:
    #get metadata 
    meta = entrada.meta
    #write shapefile output 
    with fiona.open('shape_selected_ids.shp', 'w',**meta) as output:
        for feature in entrada:
            #loop to list ids from list_ids variable
            for i in list_ids:
                #if to check selected id and write in output
                if feature['properties']['Id'] == i:
                output.write(feature)

08: Definição e leitura de extensão geométrica para cada amostra (talhão) escolhida para avaliação.


09: Definição de comando para a sobreposição entre o arquivo stack de imagens ao longo do tempo e o arquivo shapefile de amostras.

10: Definição de pasta padrão para alocação dos produtos gerados a partir da etapa anterior. Ainda nesta etapa, abertura do arquivo .tif e recorte com base nos limites geográficos de cada talhão analisado.



In [ ]:
#input path to write crops (raster subset by shapefile id),
#this is considered by the processing time of stmetrics
path_out = '/path/'

#set working output dir
os.chdir(path_out)

#open shapefile 
with fiona.open('/path/shape_selected_ids.shp', "r") as shapefile:
    for feature in shapefile:
        # obtendo a geometria associada a feição
        geom = shape(feature["geometry"])
        #print(feature)
        #open tif to crop by selected ids
        with rasterio.open('/path/lc8_2017_2019_stack_32723R.tif', "r") as src:
            out_image, out_transform = rasterio.mask.mask(src, [geom], crop=True)
            out_meta = src.meta
        #print(str(feature['properties']['Id']))
        #output file name 
        out_file = str(feature['properties']['Id']) + 'masked.tif'
        #metadata write
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        #write output in crop tif 
        with rasterio.open(out_file, "w", **out_meta) as dest:
            dest.write(out_image)

11: Listagem dos talhões escolhidos para análise dentro do conjunto de amostras disponível e recorte da série temporal de cada talhão de acordo com a extensão dos limites geográficos de cada um.

In [ ]:
#input path to cropped tifs
path = '/path/crops'

#list cropped images
listcrop = list_all_images(path)

#print list crop
print(listcrop)

12: Comando para a extração de métricas a partir dos recortes gerados.

In [ ]:
#initialize empty list variable
im_crop = []

#loop to run sits2metrics for cropped images
for crop in listcrop: 
    dataset = rasterio.open(crop)
    #metricas = numpy.vstack(lista)
    im_crop.append(stmetrics.metrics.sits2metrics(dataset))

13: Listagem das dimensões de análise e determinação da geração de figuras de cada talhão analisado em cada métrica a ser considerada.


In [ ]:
###tests to check the list of numpy arrays, stmetrics output

#check crop dimensions
#im_crop[3].ndim

#check total crop 
print(len(im_crop))

#check data type to use
print(type(im_crop))

#plot a crop to test plt.imshow
plt.imshow(im_crop[0][1,:,:])

14: Determinação das métricas consideradas, acesso à lista de imagens, ao arquivo .shp contendo as dimensões espaciais de cada talhão em linhas e colunas. Ainda nesta etapa, determinação das configurações dos arquivos de saída, inclusive das figuras geradas para cada talhão.

In [ ]:
#create header to metrics in stmetrics lib
header=["Mean", "Max", "Min", "Std", "Sum","Amplitude","First_slope","Area","Area_s1","Area_s2","Area_s3","Area_s4","Circle","Gyration","Polar_balance","Angle", "DFA","Hurst","Katz"]

#initialize figure (all) and ax with 19 rows and each crop in new column
fig, ax  = plt.subplots(19, len(im_crop) , figsize=(10,30))

#loop to create columns on lenght of cropped tifs
for column in range(1,len(im_crop)+1):
    #print(column)
    for b, n in zip(range(1, im_crop[0].shape[0]+1),header):
        #b is the metrics index - over 19 metrics
        #n is the header index  - over 19 metrics names
        #column to control the column ax -column index
        #print(column, b, n)
        # here we walk through ax row and column to plot with imshow
        ax[b-1, column-1].imshow(im_crop[column-1][b-1,:,:])
        plt.tight_layout()
        #set header title to ax
        ax[b-1, column-1].set_title(n)
#show figure
plt.show()       

#extras - to save figure
#plt.savefig(out_file +'.png', dpi=300, bbox_inches='tight')
#plt.close(fig=None)

15: Construção de listas de valores da média dos talhões para inserir em gráficos de série temporal, objetivando observar o comportamento temporal do NDVI ao longo do período selecionado.

In [ ]:
# initialize empty list variable
l = []

#for to get mean value to crop in 36 time points
for crop in listcrop:
    #initialize and clear totalcrop for an 36 time poins stack
    totalcrop = []
    #append totalcrop in a list of lists called l
    l.append(totalcrop)
    #open crop
    with rasterio.open(crop) as cropimg:
        #iterate over bands
        for b in range(1, cropimg.count+1):
            #read band
            data = cropimg.read(b)
            #calculate mean not considering no data -9999 values
            average = data[data!=-9999].mean()
            #print(b, average)
            #append the mean in totalcrop list
            totalcrop.append(average) 
            #print(b, mean)
            #create a variable num_time
            num_time = cropimg.count

16: Construção dos gráficos para os talhões ao longo do período analisado.


In [ ]:
#create a time range to use in plot
time = range(1, num_time+1)

#print(time)

#valores_cro1 = l[0]
lencrop = (len(l))


#initialize figure with crop lenght in rows
fig, ax  = plt.subplots(nrows=lencrop,ncols=1, figsize=(20,20))

#loop to create rows in lenght of cropped tifs
for b in range(0, len(l)):
    #save crop in valores crop to use in plot
    valores_crop = l[b]
    #ax plot the time and valores_crop
    ax[b].plot(time,valores_crop)
    #plt.tight_layout()
    #set header title to ax
    n = os.path.basename(listcrop[b])[0:-4]
    ax[b].set_title(n)
#show figure
plt.show()     